## Lecture 2 - $25^{th}$ November 2024
#### Lecture 2 on Variational Quantum Algorithm
<br>
This notebook presentation is part of my lecture material for the Advanced Quantum Mechanics course at the University of Trieste. It contains a general introduction to Qiskit and Pennylane

**Michele Grossi**, Coordinator of Hybrid Quantum Computing Infrastracture and Algorithms @ CERN


## Variational Quantum Algorithms

Variational Quantum Algorithms (VQA) are hybrid algorithm that use a classical optimizer to train a parameterized quantum circuit to approximate solutions for a given problem. You will find this family of architectures for a lot of application in different fields like: Chemistry, Physics, Finance, Machine Learning and more.

VQA typically need fewer gates and qubits. In turn, they are more resistant to noise and are well suited to handle near-term quantum computer constraints.

VQA's are typically iterative. Each iteration involves both quantum and classical processing. 

Output (a measurement) from one iteration is sent to the classical optimizer which generates input (a parameter) for the next iteration:

<div>
<img src="images/qva_loop.png" width="600" align="left"/>
</div>

### VQE - Application to Physics to study static properties

The Variational Quantum Eigensolver (VQE) is a central algorithm in many applications, e.g. quantum chemistry or optimization.
This tutorial shows you how to run the VQE. We'll start off by defining the algorithm settings, such as the Hamiltonian and ansatz, and then run a VQE.

In [ ]:
# VQE with PennyLane: Ground State of a 5-Qubit Hamiltonian
import pennylane as qml
from pennylane import numpy as np
from scipy.linalg import eigh

# 1. Define the problem Hamiltonian
def define_hamiltonian():
    """Construct the Hamiltonian for the VQE problem."""
    coeffs = np.random.uniform(-1, 1, 15)  # Random coefficients
    obs = [
        qml.PauliZ(0) @ qml.PauliZ(1),
        qml.PauliZ(1) @ qml.PauliZ(2),
        qml.PauliZ(2) @ qml.PauliZ(3),
        qml.PauliZ(3) @ qml.PauliZ(4),
        qml.PauliZ(0) @ qml.PauliZ(4),
        qml.PauliX(0),
        qml.PauliX(1),
        qml.PauliX(2),
        qml.PauliX(3),
        qml.PauliX(4),
        qml.PauliY(0) @ qml.PauliY(1),
        qml.PauliY(2) @ qml.PauliY(3),
        qml.PauliY(3) @ qml.PauliY(4),
        qml.PauliY(0) @ qml.PauliY(4),
        qml.PauliZ(1) @ qml.PauliZ(3),
    ]
    return qml.Hamiltonian(coeffs, obs)

hamiltonian = define_hamiltonian()
n_qubits = 5

In [ ]:
# Classical solution for comparison
def classical_ground_state(hamiltonian):
    """Compute the ground state energy of the Hamiltonian classically."""
    hamiltonian_matrix = qml.matrix(hamiltonian)  # Get the matrix representation
    eigenvalues, _ = eigh(hamiltonian_matrix)     # Diagonalize the matrix
    return min(eigenvalues)

# Compute classical ground state
classical_solution = classical_ground_state(hamiltonian)
print(f"Classical ground state energy: {classical_solution:.6f}")

In [ ]:
# 3. Define the VQE quantum circuit
dev = qml.device("default.qubit", wires=n_qubits)

def variational_ansatz(params):
    """Define the variational ansatz."""
    for i in range(n_qubits):
        qml.RX(params[i], wires=i)
        qml.RY(params[i + n_qubits], wires=i)
    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i + 1])
    qml.CNOT(wires=[n_qubits - 1, 0])  # Wrap around


# Define the QNode
@qml.qnode(dev)
def circuit(params):
    variational_ansatz(params)
    return qml.expval(hamiltonian)
    

# Generate random parameters
np.random.seed(42)
params = np.random.uniform(0, 2 * np.pi, 2 * n_qubits)

# Draw the circuit
qml.draw_mpl(circuit)(params)

In [ ]:
# 4. Optimize the VQE
np.random.seed(42)
init_params = np.random.uniform(0, 2 * np.pi, 2 * n_qubits)

optimizer = qml.GradientDescentOptimizer(stepsize=0.4)
params = init_params
max_iter = 100
convergence_tol = 1e-6

energy = []
for n in range(max_iter):
    params, cost = optimizer.step_and_cost(circuit, params)
    energy.append(cost)
    print(f"Iteration {n+1}: Energy = {cost:.6f}")
    if len(energy) > 2 and np.abs(energy[-1] - energy[-2]) < convergence_tol:
        break

print(f"VQE estimated ground state energy: {cost:.6f}")


In [ ]:
# 5. Compare classical and VQE results
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(energy, label="VQE Energy", marker="o")
plt.axhline(y=classical_solution, color="r", linestyle="--", label="Classical Energy")
plt.xlabel("Iteration")
plt.ylabel("Energy")
plt.title("VQE Optimization Progress")
plt.legend()
plt.grid()
plt.show()


#### Discussion about Pennylane return functions
PennyLane offers various return types to extract different kinds of information from quantum circuits. The difference between `qml.state()` and `qml.expval(hamiltonian)` lies in the type of information returned and its intended use.

1. qml.state()

* Purpose:
    Returns the full quantum state (statevector) of the system after executing the circuit.
* Output:
    A complex-valued array representing the amplitudes of the quantum state.
    Useful for inspecting or debugging the complete state of the system.
* Usage:
    Typically used in simulations, where the statevector is accessible.
    When you need detailed information about the full quantum state.
    Ideal for debugging or verifying intermediate results in simulations.
  

In [ ]:
@qml.qnode(dev) 
   def circuit(params):
    variational_ansatz(params)
    return qml.state()

If the system is in a state $|\psi >  = \alpha|0> + \beta |1>$  calling `qml.state()` would return the vector $[\alpha, \beta] $

If the system is in a state $|\psi >  = \alpha|0> + \beta |1>$  calling `qml.state()` would return the vector $[\alpha, \beta] $

2. qml.expval(hamiltonian)

* Purpose:
Computes the expectation value of a given observable (e.g., a Hamiltonian) with respect to the final quantum state.
* Output:
A scalar value representing the expected measurement outcome of the observable.
This is the average value you would obtain if you repeatedly measured the observable on the prepared state.
* Usage:
Used in tasks like Variational Quantum Eigensolvers (VQE) to estimate the ground state energy of a Hamiltonian.
Suitable for tasks involving real quantum hardware or noisy simulators.

In [ ]:
@qml.qnode(dev)
def circuit(params):
    variational_ansatz(params)
    return qml.expval(hamiltonian)


If the system is in state $ |\psi >$ and the Hamiltonian is $H$ the expectation value is computed as $ <\psi | H | \psi >$

### Qiskit VQE

**Note:** You can find tutorials on solving more comprehensive problems, such as finding the ground state of the lithium hydride molecule, using the VQE within [the tutorials of Qiskit Nature](https://qiskit-community.github.io/qiskit-nature/tutorials/index.html).

we want to use a variational algorithm to find the eigenvalue of the following observable:

$O_1 = 2II−2XX+3YY−3ZZ$ 

This observable has the following eigenvalues: $\lambda_0 = -6, \lambda_1 = 4, \lambda_2 = 4, \lambda_3 = 6$

In [ ]:
from qiskit.quantum_info import SparsePauliOp

observable_1 = SparsePauliOp.from_list([("II", 2), ("XX", -2), ("YY", 3), ("ZZ", -3)])

### Building VQE from scratch

We'll first explore how to construct a VQE instance manually to find the lowest eigenvalue for $O_1$.

In [ ]:
def cost_func_vqe(params, ansatz, hamiltonian, estimator):
    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (Estimator): Estimator primitive instance

    Returns:
        float: Energy estimate
    """
    pub = (ansatz, hamiltonian, params)
    cost = estimator.run([pub]).result()[0].data.evs

    return cost

In [ ]:
from qiskit.circuit.library import TwoLocal
from qiskit import QuantumCircuit

import numpy as np

reference_circuit = QuantumCircuit(2)
reference_circuit.x(0)

variational_form = TwoLocal(
    2,
    rotation_blocks=["rz", "ry"],
    entanglement_blocks="cx",
    entanglement="linear",
    reps=1,
)
raw_ansatz = reference_circuit.compose(variational_form)

raw_ansatz.decompose().draw('mpl')

In [ ]:
## Let's start on local simulators

from qiskit.primitives import StatevectorEstimator as Estimator
from qiskit.primitives import StatevectorSampler as Sampler
estimator = Estimator()
sampler = Sampler()

In [ ]:
## initial parameters 
import numpy as np

x0 = np.ones(raw_ansatz.num_parameters)
print(x0)

We can minimize this cost function to calculate optimal parameters

In [ ]:
# SciPy minimizer routine
from scipy.optimize import minimize
import time

start_time = time.time()

result = minimize(cost_func_vqe, x0, args=(raw_ansatz, observable_1, estimator), method="COBYLA", options={'maxiter': 1000, 'disp': True})

end_time = time.time()
execution_time = end_time - start_time

In [ ]:
result

we can check this by using NumPy's linear algebra eigensolver due to the small dimension of the problem.

In [ ]:
from numpy.linalg import eigvalsh

solution_eigenvalue = min(eigvalsh(observable_1.to_matrix()))

print(f"""Number of iterations: {result.nfev}""")
print(f"""Time (s): {execution_time}""")

print(
    f"Percent error: {abs((result.fun - solution_eigenvalue)/solution_eigenvalue):.2e}"
)

#### Experimenting with different optimizers
We can adjust the optimizer using SciPy `minimize`'s  method  argument, with more options found here. We originally used a constrained minimizer (  `COBYLA`  ). In this example, we'll explore using an unconstrained minimizer (  `BFGS`  ) instead

In [ ]:
import time
start_time = time.time()

result = minimize(cost_func_vqe, x0, args=(raw_ansatz, observable_1, estimator), method="BFGS")

end_time = time.time()
execution_time = end_time - start_time

In [ ]:
print("CHANGED TO BFGS OPTIMIZER:")
print(f"""Number of iterations: {result.nfev}""")
print(f"""Time (s): {execution_time}""")

#### VQE with Qiskit Primitives

In [ ]:
from qiskit.primitives import Estimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA, L_BFGS_B, SLSQP
from qiskit_algorithms.utils import algorithm_globals
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import SparsePauliOp

H2_op = SparsePauliOp.from_list(
    [
        ("II", -1.052373245772859),
        ("IZ", 0.39793742484318045),
        ("ZI", -0.39793742484318045),
        ("ZZ", -0.01128010425623538),
        ("XX", 0.18093119978423156),
    ]
)

In [ ]:
estimator = Estimator()


# we will iterate over these different optimizers
optimizers = [COBYLA(maxiter=80), L_BFGS_B(maxiter=60), SLSQP(maxiter=60)]
converge_counts = np.empty([len(optimizers)], dtype=object)
converge_vals = np.empty([len(optimizers)], dtype=object)

for i, optimizer in enumerate(optimizers):
    print("\rOptimizer: {}        ".format(type(optimizer).__name__), end="")
    algorithm_globals.random_seed = 50
    ansatz = TwoLocal(rotation_blocks="ry", entanglement_blocks="cz")

    counts = []
    values = []

    def store_intermediate_result(eval_count, parameters, mean, std):
        counts.append(eval_count)
        values.append(mean)

    vqe = VQE(estimator, ansatz, optimizer, callback=store_intermediate_result)
    #result = vqe.compute_minimum_eigenvalue(operator=H2_op)
    result = vqe.compute_minimum_eigenvalue(operator=observable_1)
    converge_counts[i] = np.asarray(counts)
    converge_vals[i] = np.asarray(values)

print("\rOptimization complete      ")

Now, from the callback data you stored, you can plot the energy value at each objective function call each optimizer makes. An optimizer using a finite difference method for computing gradient has that characteristic step-like plot where for a number of evaluations it is computing the value for close by points to establish a gradient 

In [ ]:
import pylab

pylab.rcParams["figure.figsize"] = (12, 8)
for i, optimizer in enumerate(optimizers):
    pylab.plot(converge_counts[i], converge_vals[i], label=type(optimizer).__name__)
pylab.xlabel("Eval count")
pylab.ylabel("Energy")
pylab.title("Energy convergence for various optimizers")
pylab.legend(loc="upper right");

### Discussion on the ansatz choice
To iteratively optimize from a reference state $\rho$ to a target state $\psi(\theta)$, we need to define a variational form $U_V(\theta)$ that represents a collection of parametrized states for our variational algorithm to explore:

$U_V(\theta) U_R | 0> = U_V(\theta) \rho= U_A(\theta) = |\psi(\theta)> $

Note that the parametrized state depends on both the reference state $\rho$, which does not depend on any parameters, and the variational form $U_V(\theta)$, which always depends on parameters. We refer to the combination of these two halves as an ansatz $U_A(\theta)$. As we construct our ansatz to represent a collection of parametrized states for our variational algorithm to explore, we realize an important issue: dimensionality. An 
n-qubit system (i.e., Hilbert space) has a vast number of distinct quantum states in the configuration space. We would require an unwieldy number of parameters to fully explore it. In addition, the runtime complexity of search algorithms, and others alike, grows exponentially with this dimensionality, a phenomenon often referred to in the literature as the curse of dimensionality. 

To counter this setback, it is common practice to impose some reasonable constraints on the variational form such that only the most relevant states are explored. Finding an efficient truncated ansatz is an active area of research, but we'll cover two common designs.

#### Heuristic ansatze and trade-offs

If you do not have any information about your particular problem that can help restrict the dimensionality, you can try an arbitrary family of parameterized circuits with fewer than $2^{2n}$ parameters. However, there are some trade-offs to consider:

- Speed: By reducing the search space, the algorithm can run faster.
- Accuracy: However, reducing the space could risk excluding the actual solution to the problem, leading to suboptimal solutions.
- Noise: Deeper circuits are affected by noise, so we need to experiment with our ansatz's connectivity, gates, and gate fidelity.

**N-Local circuit**

One of the most widely used examples of heuristic ansatzes is the N-local circuits, for a few reasons:

- Efficient implementation: The N-local ansatz is typically composed of simple, local gates that can be implemented efficiently on a quantum computer, using a small number of physical qubits. This makes it easier to construct and optimize quantum circuits.
- Captures important correlations: The N-local ansatz can capture important correlations between the qubits in a quantum system, even with a small number of gates. This is because the local gates can act on neighboring qubits and create entanglement between them, which can be important for simulating complex quantum systems.

These circuits consist of rotation and entanglement layers that are repeated alternatively one or more times as follows:
- Each layer is formed by gates of size at most N, where N has to be lower than the number of qubits.
- For a rotation layer, the gates are stacked on top of each other. We can use standard rotation operations (RX.RZ)
- For an entanglement layer, we can use gates like Toffoligates or  CX  with an entanglement strategy.

  
Example: 5 qubits with Toffoli gate, which acts on three qubits, making the circuit `3-local`

In [ ]:
from qiskit.circuit.library import NLocal, CCXGate, CRZGate, RXGate
from qiskit.circuit import Parameter

theta = Parameter("θ")
ansatz = NLocal(
    num_qubits=5,
    rotation_blocks=[RXGate(theta), CRZGate(theta)],
    entanglement_blocks=CCXGate(),
    entanglement=[[0, 1, 2], [0, 2, 3], [4, 2, 1], [3, 1, 0]],
    reps=2,
    insert_barriers=True,
)
ansatz.decompose().draw("mpl")

In [ ]:
## Example using the standard 2-local circuits with single-qubit rotation gates and 2-qubit entanglement gates.
from qiskit.circuit.library import TwoLocal

ansatz = TwoLocal(
    num_qubits=5,
    rotation_blocks=["rx", "rz"],
    entanglement_blocks="cx",
    entanglement="linear",
    reps=2,
    insert_barriers=True,
)
ansatz.decompose().draw("mpl")

**EfficientSU2**

EfficientSU2  is a hardware-efficient circuit that consists of layers of single-qubit operations spanning $SU(2)$ and  $CX$ entanglements. This is a heuristic pattern that can be used to prepare trial wave functions for variational quantum algorithms or as a classification circuit for machine learning.

In [ ]:
from qiskit.circuit.library import EfficientSU2

ansatz = EfficientSU2(4, su2_gates=["rx", "y"], entanglement="linear", reps=1)
ansatz.decompose().draw("mpl")